In [6]:
"""
Bayesian Two‑Stage Stochastic Portfolio Optimisation
– implements exactly the research workflow we discussed
————————————————————————————————————————————
• Part‑A parameters (μ̂ & Σ̂) hard‑coded
• 8 scenarios = up/down on the three risky assets
• simulate 10 y returns under a chosen “true” scenario
• Bayesian update → posterior π₁₀
• build & solve a two‑stage stochastic LP (shorting allowed)
————————————————————————————————————————————
"""

import numpy as np
import pandas as pd
from scipy.optimize import linprog

# ───────────────────────────────────────────────
# 0. Helper pretty‑print
# ───────────────────────────────────────────────
def print_bar(title):
    print("\n" + "─" * 60)
    print(title)
    print("─" * 60)

# ───────────────────────────────────────────────
# 1. Part‑A statistical parameters
# ───────────────────────────────────────────────
assets = ["Stocks", "Bonds", "Commodities", "Cash"]
mu_hat = np.array([0.038, 0.050, 0.113, 0.001])       # annual expected returns
Sigma_hat = np.array([[ 0.070, -0.007,  0.015,  0.003],
                      [-0.007,  0.033, -0.012,  0.001],
                      [ 0.015, -0.012,  0.098, -0.001],
                      [ 0.003,  0.001, -0.001,  0.001]])

# ───────────────────────────────────────────────
# 2. Build eight scenarios
# ───────────────────────────────────────────────
delta = 0.02                       # ↑/↓ shift in expected return (±2 pp)
states = [(e, b, c)                # 1 = up, −1 = down
          for e in [1, -1]
          for b in [1, -1]
          for c in [1, -1]]
labels = [f"({e:+},{b:+},{c:+})" for (e, b, c) in states]

def scenario_mu(e, b, c):
    mu = mu_hat.copy()
    mu[0] += e * delta             # Stocks
    mu[1] += b * delta             # Bonds
    mu[2] += c * delta             # Commodities
    return mu

mu_scen = np.array([scenario_mu(*abc) for abc in states])  # shape (8,4)

print_bar("Scenario Mean Return Matrix (annual)")
display(pd.DataFrame(mu_scen, index=labels, columns=assets))

# ───────────────────────────────────────────────
# 3. Simulate 10 years of returns under a “true” scenario
# ───────────────────────────────────────────────
np.random.seed(42)
true_scen_id = 0                          # choose scenario 0 == (+,+,+)
T = 10

returns_path = np.random.multivariate_normal(
    mean=mu_scen[true_scen_id], cov=Sigma_hat, size=T
)
print_bar("Simulated 10‑Year Annual Returns (sample)")
display(pd.DataFrame(returns_path, columns=assets).head())

# ───────────────────────────────────────────────
# 4. Bayesian posterior after 10 years
# ───────────────────────────────────────────────
inv_Sigma = np.linalg.inv(Sigma_hat)

def log_mvn_pdf(r, mean):
    diff = r - mean
    return -0.5 * diff @ inv_Sigma @ diff   # constant term cancels in Bayes ratio

log_like = np.zeros(8)
for s in range(8):
    for r in returns_path:
        log_like[s] += log_mvn_pdf(r, mu_scen[s])
# stabilise
log_like -= log_like.max()
weights = np.exp(log_like)
posterior = weights / weights.sum()

print_bar("Posterior Probabilities π₁₀(s)")
display(pd.DataFrame({"Scenario": labels, "π10": posterior}))

# ───────────────────────────────────────────────
# 5. Build & solve the two‑stage stochastic LP
# ───────────────────────────────────────────────
V0 = 1.0               # normalise initial wealth
S10 = 0.0              # no extra deposit

# deterministic 10‑y factors:  (1+μ)^10 − 1  ≈ cumulative return
R0 = (1 + mu_scen) ** 10 - 1
R1 = R0.copy()         # same expectation for 2nd decade

# Decision variables: x0 (4) + x1 (8×4)
n_var = 4 + 8 * 4

# objective coefficients   c_i = π(s)*(1+R1)
c = np.zeros(n_var)
for s in range(8):
    for i in range(4):
        idx = 4 + s * 4 + i
        c[idx] = posterior[s] * (1 + R1[s, i])
# maximise → minimise negative
c_min = -c

# Equality constraints
A_eq = []
b_eq = []

# (i) initial budget
row = np.zeros(n_var)
row[:4] = 1
A_eq.append(row)
b_eq.append(V0)

# (ii) scenario budgets
for s in range(8):
    row = np.zeros(n_var)
    # Σ x1 = Σ x0 * (1+R0)
    row[4 + s * 4 : 4 + s * 4 + 4] = 1
    row[:4] -= (1 + R0[s])
    A_eq.append(row)
    b_eq.append(S10)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds: allow shorting (wide range to avoid unbounded problem explosion)
bounds = [(-10, 10)] * n_var

res = linprog(c_min, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

print_bar("Solver Status")
print(res.message)

x0_opt = res.x[:4]
x1_opt = res.x[4:].reshape(8, 4)
exp_final_wealth = -res.fun

print_bar("Optimal Initial Portfolio x0*")
display(pd.Series(x0_opt, index=assets))

print_bar("Scenario‑Specific Rebalancing x1*")
df_x1 = pd.DataFrame(x1_opt, index=labels, columns=assets)
display(df_x1)  # show first few

print_bar(f"Expected Final Wealth  E[V₍₂₀₎]  = {exp_final_wealth:,.2f}  (per $1 initial wealth)")




────────────────────────────────────────────────────────────
Scenario Mean Return Matrix (annual)
────────────────────────────────────────────────────────────


,Stocks,Bonds,Commodities,Cash
"(+1,+1,+1)",0.058,0.07,0.133,0.001
"(+1,+1,-1)",0.058,0.07,0.093,0.001
"(+1,-1,+1)",0.058,0.03,0.133,0.001
"(+1,-1,-1)",0.058,0.03,0.093,0.001
"(-1,+1,+1)",0.018,0.07,0.133,0.001
"(-1,+1,-1)",0.018,0.07,0.093,0.001
"(-1,-1,+1)",0.018,0.03,0.133,0.001
"(-1,-1,-1)",0.018,0.03,0.093,0.001



────────────────────────────────────────────────────────────
Simulated 10‑Year Annual Returns (sample)
────────────────────────────────────────────────────────────


,Stocks,Bonds,Commodities,Cash
0,0.009995,-0.014194,-0.043783,0.041024
1,0.109803,-0.217693,0.136653,0.014666
2,0.004560,0.127752,0.339388,-0.015512
3,0.500407,0.358674,-0.092265,0.020874
4,0.137893,0.169668,0.486714,-0.036100



────────────────────────────────────────────────────────────
Posterior Probabilities π₁₀(s)
────────────────────────────────────────────────────────────


...


────────────────────────────────────────────────────────────
Solver Status
────────────────────────────────────────────────────────────
Optimization terminated successfully. (HiGHS Status 7: Optimal)

────────────────────────────────────────────────────────────
Optimal Initial Portfolio x0*
────────────────────────────────────────────────────────────


Stocks         -9.0
Bonds          10.0
Commodities    10.0
Cash          -10.0
dtype: float64


────────────────────────────────────────────────────────────
Scenario‑Specific Rebalancing x1*
────────────────────────────────────────────────────────────


,Stocks,Bonds,Commodities,Cash
"(+1,+1,+1)",10.000000,10.000000,10.0,-1.387300
"(+1,+1,-1)",8.088304,10.000000,10.0,-10.000000
"(+1,-1,+1)",10.000000,10.000000,10.0,-7.619650
"(+1,-1,-1)",10.000000,1.855955,10.0,-10.000000
"(-1,+1,+1)",10.000000,10.000000,10.0,3.671071
"(-1,+1,-1)",10.000000,10.000000,10.0,-6.853325
"(-1,-1,+1)",10.000000,10.000000,10.0,-2.561279
"(-1,-1,-1)",6.914325,10.000000,10.0,-10.000000



────────────────────────────────────────────────────────────
Expected Final Wealth  E[V₍₂₀₎]  = 55.46  (per $1 initial wealth)
────────────────────────────────────────────────────────────
